<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=79e485ecf2a88a30f52fd32696444cbb12334447a04a21b2c5302aafbd3edd44
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-15 15:21:51
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.41 C
-------------------
Today PnL: 44.57 K (0.32%)
Current PnL: -26.00 L (-16.79%)
CY Booked + Current PnL: -11.71 L (-7.56%)
-------------------
Total profit:  1.48 L
Total loss:  -27.48 L
-------------------
Total Booked + Current PnL: 15.27 L (11.95%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.13%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 90.63 L (64.25%)
Deployed:  1.28 C
Current:  1.41 C
CAGR/XIRR %: 5.5%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.48,-17.14,20.79,0.09,17361.0,-17271.0,83506.0,93.81,40.0,M-SC,2.35,253.0,-0.99,0.59,9.71,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,0.69,-11.74,13.33,0.02,20536.0,-20501.0,154058.0,134.93,54.0,M-SC,15.37,234.0,-1.00,1.10,34.00,OX40N,NTT,PAINTS
82,VOLTAS,1530.00,0.84,8.64,10.17,19.69,21185.0,16563.0,208305.0,-1.91,57.0,X-MC,2.57,78.0,0.78,1.48,16.02,XY25,NTT,AC
50,MASFIN,398.61,1.30,0.20,21.81,22.05,21412.0,195.0,98175.0,-13.73,61.0,H-SC,7.22,164.0,0.01,0.70,42.70,XR,ATH,FINANCE
43,ITC,452.00,0.51,-1.44,12.40,10.78,29371.0,-3452.0,236866.0,-41.15,46.0,X-LC,1.91,5.0,-0.12,1.69,3.72,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COFFEEDAY,80.00,12.46,-38.05,118.34,35.27,83251.0,-43200.0,70349.0,-51.71,53.0,L-SC,10.81,270.0,-0.52,0.50,72.18,XR,NTT,HOTELS
57,RAJOOENG,143.10,6.98,-33.76,110.75,39.61,75199.0,-34600.0,67900.0,-55.26,38.0,H-SC,28.03,135.0,-0.46,0.48,6.98,AR,ATH,MISC
44,JCHAC,2282.00,3.97,-34.98,53.77,-0.01,44685.0,-44701.0,83104.0,16900.00,34.0,M-SC,6.61,233.0,-1.00,0.59,6.36,OX40N,NTT,AC
0,5PAISA,593.00,2.78,-39.74,87.36,12.90,115022.0,-86832.0,131664.0,93.51,49.0,H-SC,12.02,161.0,-0.75,0.94,9.48,OX40N,NTT,FINANCE
22,DIXON,18931.72,2.59,-8.60,37.94,26.07,67694.0,-16796.0,178425.0,-55.55,42.0,X-MC,8.32,56.0,-0.25,1.27,11.12,X40N,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.0,-3.43,-18.63,123.27,81.67,94094.0,-17478.0,76332.0,6855.56,39.0,L-SC,12.90,271.0,-0.19,0.54,47.82,XR,NTT,CERAMICS
80,VALIANTORG,838.0,-2.62,-44.47,213.62,74.15,194069.0,-72757.0,90848.0,-263.44,45.0,H-SC,22.29,149.0,-0.37,0.65,16.45,XR,NTT,CHEMICALS
65,SIEMENS,4671.5,-1.62,-15.51,48.56,25.51,76351.0,-28865.0,157230.0,-3.51,44.0,H-LC,4.23,49.0,-0.38,1.12,15.87,AR,ATH,ELECTRICAL
16,CERA,9475.0,-1.54,-32.30,83.00,23.89,98838.0,-56821.0,119082.0,-34.61,27.0,H-SC,10.29,157.0,-0.57,0.85,2.84,OX40N,NTT,CERAMICS
28,GREENPANEL,537.0,-1.26,-37.34,128.45,43.14,147344.0,-68369.0,114709.0,157.11,33.0,M-SC,9.84,240.0,-0.46,0.82,7.90,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,398.61,1.3,0.2,21.81,22.05,21412.0,195.0,98175.0,-13.73,61.0,H-SC,7.22,164.0,0.01,0.7,42.7,XR,ATH,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,2.28,-4.12,116.85,107.91,165412.0,-6085.0,141559.0,-22.17,51.0,M-SC,12.60,216.0,-0.04,1.01,5.55,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.69,-11.74,13.33,0.02,20536.0,-20501.0,154058.0,134.93,54.0,M-SC,15.37,234.0,-1.00,1.10,34.00,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.48,-17.14,20.79,0.09,17361.0,-17271.0,83506.0,93.81,40.0,M-SC,2.35,253.0,-0.99,0.59,9.71,OX40N,NTT,DURABLES
66,SIS,528.00,1.60,-24.20,59.98,21.26,50481.0,-26869.0,84163.0,1985.49,50.0,H-SC,4.86,166.0,-0.53,0.60,13.71,OX40N,NTT,MISC
47,KANSAINER,340.00,1.33,-24.42,50.13,13.47,102176.0,-65844.0,203823.0,-68.90,38.0,H-SC,4.74,158.0,-0.64,1.45,3.64,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.46,7.37,60.43,72.27,109949.0,12496.0,181944.0,-7.89,70.0,M-LC,2.22,99.0,0.11,1.3,14.01,XR,NTT,IT
50,MASFIN,398.61,1.30,0.20,21.81,22.05,21412.0,195.0,98175.0,-13.73,61.0,H-SC,7.22,164.0,0.01,0.7,42.70,XR,ATH,FINANCE


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.46,7.37,60.43,72.27,109949.0,12496.0,181944.0,-7.89,70.0,M-LC,2.22,99.0,0.11,1.30,14.01,XR,NTT,IT
50,MASFIN,398.61,1.30,0.20,21.81,22.05,21412.0,195.0,98175.0,-13.73,61.0,H-SC,7.22,164.0,0.01,0.70,42.70,XR,ATH,FINANCE
38,INDIAMART,4810.62,1.43,-2.71,112.48,106.72,134967.0,-3344.0,119992.0,-51.77,38.0,H-SC,2.67,139.0,-0.02,0.85,20.03,AR,ATH,MISC
25,FINCABLES,1641.55,2.28,-4.12,116.85,107.91,165412.0,-6085.0,141559.0,-22.17,51.0,M-SC,12.60,216.0,-0.04,1.01,5.55,OX40N,ATH,CABLES
1,ABB,7934.00,0.09,-3.08,50.19,45.57,127262.0,-8059.0,253560.0,-37.68,60.0,H-MC,4.58,120.0,-0.06,1.81,7.89,AR,NTT,ELECTRICAL


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.55,-18.54,90.85,55.47,180054.0,-45098.0,198188.0,-27.82,23.0,X-MC,12.48,64.0,-0.25,1.41,1.55,X40N,ATH,FINANCE
75,TMPV,600.00,-0.19,-28.92,73.01,22.98,117231.0,-65330.0,160568.0,-26.74,30.0,X-LC,6.47,3.0,-0.56,1.14,0.99,XY24,NTT,AUTO
53,PAGEIND,51605.08,-0.45,-8.73,40.36,28.11,74192.0,-17585.0,183825.0,-33.88,31.0,X-MC,12.66,55.0,-0.24,1.31,0.19,X40,ATH,APPARELS
23,DMART,5391.45,-0.46,-8.05,40.94,29.60,93965.0,-20082.0,229518.0,-21.58,32.0,X-LC,5.49,19.0,-0.21,1.63,12.38,X40N,ATH,FMCG
2,ABBOTINDIA,35195.00,-0.39,-6.40,26.15,18.07,43775.0,-11455.0,167400.0,-19.83,32.0,X-MC,5.48,63.0,-0.26,1.19,10.93,X40,ATH,PHARMA


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,-0.00,-1.05,25.79,24.48,64273.0,-2640.0,249216.0,-81.09,44.0,X-SC,0.69,86.0,-0.04,1.78,21.85,X40N,NTT,MISC
43,ITC,452.0,0.51,-1.44,12.40,10.78,29371.0,-3452.0,236866.0,-41.15,46.0,X-LC,1.91,5.0,-0.12,1.69,3.72,X40,NTT,FMCG
36,ICICIPRULI,790.0,0.30,7.48,21.63,30.73,45799.0,14732.0,211737.0,-17.71,65.0,X-MC,2.15,75.0,0.32,1.51,21.20,X40,ATH,INSURANCE
33,HINDUNILVR,2922.0,1.55,-10.70,27.29,13.67,62647.0,-27503.0,229560.0,-17.44,34.0,X-LC,2.18,14.0,-0.44,1.64,8.37,XY25,NTT,FMCG
3,ACC,3906.0,0.40,-25.25,119.60,64.15,212733.0,-60081.0,177870.0,-56.31,36.0,X-SC,2.42,82.0,-0.28,1.27,0.40,XY24,BTT,CEMENT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,PAGEIND,51605.08,-0.45,-8.73,40.36,28.11,74192.0,-17585.0,183825.0,-33.88,31.0,X-MC,12.66,55.0,-0.24,1.31,0.19,X40,ATH,APPARELS
3,ACC,3906.00,0.40,-25.25,119.60,64.15,212733.0,-60081.0,177870.0,-56.31,36.0,X-SC,2.42,82.0,-0.28,1.27,0.40,XY24,BTT,CEMENT
75,TMPV,600.00,-0.19,-28.92,73.01,22.98,117231.0,-65330.0,160568.0,-26.74,30.0,X-LC,6.47,3.0,-0.56,1.14,0.99,XY24,NTT,AUTO
11,BATAINDIA,2096.00,-0.06,-38.09,118.40,35.20,94312.0,-49015.0,79655.0,4.40,34.0,X-SC,15.98,92.0,-0.52,0.57,1.18,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,-0.55,-18.54,90.85,55.47,180054.0,-45098.0,198188.0,-27.82,23.0,X-MC,12.48,64.0,-0.25,1.41,1.55,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,0.58,-29.01,101.21,42.84,46706.0,-18858.0,46148.0,-53.72,45.0,X-SC,35.90,83.0,-0.40,0.33,4.40,XY24,NTT,MISC
59,RELAXO,1176.00,-0.19,-47.64,190.87,52.31,145076.0,-69152.0,76008.0,-44.25,42.0,X-SC,6.72,91.0,-0.48,0.54,1.74,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.06,-38.09,118.40,35.20,94312.0,-49015.0,79655.0,4.40,34.0,X-SC,15.98,92.0,-0.52,0.57,1.18,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-0.40,-4.16,29.47,24.08,35855.0,-5283.0,121667.0,-11.91,33.0,X-SC,5.22,89.0,-0.15,0.87,14.64,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,0.46,-16.38,71.39,43.32,97233.0,-26672.0,136200.0,-28.18,37.0,X-SC,4.62,90.0,-0.27,0.97,4.58,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.48,-11.02,35.66,20.71,120015.0,-41683.0,336554.0,-23.30,66.0,X-LC,4.66,1.0,-0.35,2.40,12.46,X40,ATH,IT
41,INFY,1972.00,0.49,9.33,22.79,34.24,77960.0,29190.0,342078.0,-14.93,64.0,X-LC,3.59,2.0,0.37,2.44,18.38,X40,NTT,IT
75,TMPV,600.00,-0.19,-28.92,73.01,22.98,117231.0,-65330.0,160568.0,-26.74,30.0,X-LC,6.47,3.0,-0.56,1.14,0.99,XY24,NTT,AUTO
81,VBL,671.64,-0.52,-3.56,40.67,35.67,123886.0,-11229.0,304613.0,-14.99,56.0,X-LC,4.48,4.0,-0.09,2.17,9.81,X40N,ATH,FMCG
43,ITC,452.00,0.51,-1.44,12.40,10.78,29371.0,-3452.0,236866.0,-41.15,46.0,X-LC,1.91,5.0,-0.12,1.69,3.72,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.48,-37.23,111.71,32.89,54136.0,-28746.0,48461.0,-698.49,56.0,L-MC,5.98,259.0,-0.53,0.35,33.46,XR,NTT,BANKS
6,ASIANTILES,137.00,-3.43,-18.63,123.27,81.67,94094.0,-17478.0,76332.0,6855.56,39.0,L-SC,12.90,271.0,-0.19,0.54,47.82,XR,NTT,CERAMICS
50,MASFIN,398.61,1.30,0.20,21.81,22.05,21412.0,195.0,98175.0,-13.73,61.0,H-SC,7.22,164.0,0.01,0.70,42.70,XR,ATH,FINANCE
13,BSOFT,831.70,1.07,-21.26,91.50,50.79,101333.0,-29907.0,110746.0,0.89,69.0,H-SC,6.09,151.0,-0.30,0.79,28.95,XR,ATH,IT
39,INDIGOPNTS,1408.00,0.69,-11.74,13.33,0.02,20536.0,-20501.0,154058.0,134.93,54.0,M-SC,15.37,234.0,-1.00,1.10,34.00,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,1.07,-21.26,91.50,50.79,101333.0,-29907.0,110746.0,0.89,69.0,H-SC,6.09,151.0,-0.30,0.79,28.95,XR,ATH,IT
84,WIPRO,420.00,0.46,7.37,60.43,72.27,109949.0,12496.0,181944.0,-7.89,70.0,M-LC,2.22,99.0,0.11,1.30,14.01,XR,NTT,IT
41,INFY,1972.00,0.49,9.33,22.79,34.24,77960.0,29190.0,342078.0,-14.93,64.0,X-LC,3.59,2.0,0.37,2.44,18.38,X40,NTT,IT
1,ABB,7934.00,0.09,-3.08,50.19,45.57,127262.0,-8059.0,253560.0,-37.68,60.0,H-MC,4.58,120.0,-0.06,1.81,7.89,AR,NTT,ELECTRICAL
50,MASFIN,398.61,1.30,0.20,21.81,22.05,21412.0,195.0,98175.0,-13.73,61.0,H-SC,7.22,164.0,0.01,0.70,42.70,XR,ATH,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.31
1,25,44.89
2,50,76.68


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.03
MC    30.94
LC    24.07
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.80
X40      22.98
X40N     13.01
XR        9.82
XY25      9.08
AR        9.05
OX40N     7.54
SR        0.98
MH        0.78
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.43
H-SC    23.88
X-LC    20.64
M-SC    11.68
X-SC     8.06
H-MC     4.82
L-SC     1.41
M-MC     1.34
M-LC     1.30
H-LC     1.12
L-LC     1.01
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.34,-7.20,41.67
IT,13.04,-14.92,72.71
FINANCE,9.56,-18.75,70.40
MISC,7.31,-26.74,78.58
ELECTRICAL,6.01,-11.25,51.99
PAINTS,5.85,-12.40,29.10
INSURANCE,4.79,-0.91,35.44
PHARMA,4.11,-1.78,33.82
AUTO,2.81,-32.77,78.79


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3162146.0,21
AR,1302940.0,10
XR,1285786.0,13
X40,997309.0,14
X40N,859289.0,9
OX40N,733066.0,10
XY25,365588.0,6
SR,281711.0,2
MH,75151.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3627927.0,25
M-SC,1419906.0,15
X-MC,1397255.0,16
X-LC,829300.0,11
X-SC,788836.0,8
H-MC,402398.0,3
L-SC,262710.0,3
M-LC,109949.0,1
H-LC,76351.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1264303.0      6
           AR         892937.0      5
M-SC       XY24       815885.0      6
H-SC       XR         771228.0      7
X-MC       X40        485291.0      7
           XY24       399740.0      3
H-SC       OX40N      342597.0      4
X-MC       X40N       334856.0      4
X-LC       X40        325058.0      5
X-SC       X40N       306582.0      3
M-SC       OX40N      305104.0      5
X-SC       XY24       295294.0      3
H-SC       SR         281711.0      2
X-LC       X40N       217851.0      2
H-MC       AR         207863.0      2
           XY24       194535.0      1
X-LC       XY24       192389.0      2
X-SC       X40        186960.0      2
X-MC       XY25       177368.0      2
L-SC       XR         177345.0      2
M-SC       XR         173128.0      2
           AR         125789.0      2
M-LC       XR         109949.0      1
X-LC       XY25        94002.0      2
L-SC       OX40N       85365.0      1
H-LC       AR          76351.0      1
H-SC       MH          75151.0      1
M-MC       XY25        57044.0      1
L-MC       XR          54136.0      1
L-LC       XY25        37174.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 24.0 seconds
